# [Day 7](https://adventofcode.com/2023/day/7)


In [1]:
import pandas as pd
from collections import Counter

In [2]:
# Read inputs
isTest = not True
fileName = "test" if isTest else "input"
df = pd.read_csv(fileName, delimiter=" ", names=["hand", "bid"])
df

,hand,bid
0,398KA,456
1,2J299,282
2,8939K,547
3,9TAA9,79
4,47TJ4,431
...,...,...
995,TJKQK,260
996,44945,374
997,K2AT3,313
998,33363,311


In [3]:
card = {v: k for k, v in enumerate(list("__23456789TJQKA"))}
del card["_"]
types = {
    v: k
    for k, v in enumerate(
        list(["hight", "pair_1", "pair_2", "three", "full", "four", "five"])
    )
}
card.keys(), types.keys()

(dict_keys(['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']),
 dict_keys(['hight', 'pair_1', 'pair_2', 'three', 'full', 'four', 'five']))

In [4]:
def get_high_card(hand: str):
    l = list(hand)
    return pd.DataFrame(
        {
            "card_name": l,
            "card_val": [card[c] for c in l],
        }
    ).max()["card_name"]


def get_hand_type(hand: str):
    hand_count = Counter(list(hand))
    card_count = hand_count.values()

    def get_type(card_count):
        hightest_num = max(card_count)
        num_of_pair = sum(pd.Series(card_count) == 2)
        match hightest_num:
            case 5:
                return "five"
            case 4:
                return "four"
            case 3:
                return "full" if num_of_pair > 0 else "three"
            case 2:
                return "pair_2" if num_of_pair > 1 else "pair_1"
            case _:
                return "hight"

    return get_type(card_count)

def convert_hand_to_hex(hand: str):
  res = 0
  for c in hand:
    res <<= 4
    res += card[c]
  return res

def get_card_score(hand: str, hand_type: str):
    # hand_count = pd.DataFrame(Counter(list(hand)))
    card_score = convert_hand_to_hex(hand)
    type_score = 1_000_000 * types[hand_type]
    return card_score + type_score


_l = [
    "AAAAA",
    "AA8AA",
    "23332",
    "TTT98",
    "23432",
    "A23A4",
    "23456",
    "22222",
    "2222A",
    "3322K",
    "AA22Q",
    "AAAAA",
    "KKKKK",
    "22222",
    "2222K",
    "QQQTT",
    "QQAAT",
    "QQAAK",
    "KTJJT",
    "KK677"
]

_hand_type = list(map(get_hand_type, _l))
_high_card = list(map(get_high_card, _l))
_score_hand = list(map(get_card_score, _l, _hand_type))
pd.DataFrame(
    zip(_l, _hand_type, _high_card, _score_hand),
    columns=["hand", "type", "high", "score"],
).sort_values('score', ascending=False)

,hand,type,high,score
0,AAAAA,five,A,6978670
11,AAAAA,five,A,6978670
12,KKKKK,five,K,6908765
13,22222,five,2,6139810
7,22222,five,2,6139810
1,AA8AA,four,A,5977134
8,2222A,four,A,5139822
14,2222K,four,K,5139821
15,QQQTT,full,T,4838826
2,23332,full,3,4144178


In [5]:
df['type'] = pd.DataFrame(df.apply(lambda r: get_hand_type(r["hand"]), axis=1))
df['type_val'] = df.apply(lambda r: types.get(r['type']), axis=1)
df['high'] = pd.DataFrame(df.apply(lambda r: get_high_card(r["hand"]), axis=1))
df['high_val'] = df.apply(lambda r: card.get(r['high']), axis=1)
df['score'] = df.apply(lambda r: get_card_score(r['hand'], r['type']), axis=1)

df

,hand,bid,type,type_val,high,high_val,score
0,398KA,456,hight,0,K,13,235742
1,2J299,282,pair_2,2,J,11,2176793
2,8939K,547,pair_1,1,K,13,1562077
3,9TAA9,79,pair_2,2,T,10,2634601
4,47TJ4,431,pair_1,1,T,10,1293556
...,...,...,...,...,...,...,...
995,TJKQK,260,pair_1,1,T,10,1703949
996,44945,374,three,3,9,9,3280901
997,K2AT3,313,hight,0,T,10,863907
998,33363,311,four,5,6,6,5209763


In [6]:
sorted_df = df.sort_values('score')
sorted_df

,hand,bid,type,type_val,high,high_val,score
146,23J56,453,hight,0,J,11,146262
156,265K8,511,hight,0,K,13,157144
672,26957,462,hight,0,9,9,158039
61,2785J,351,hight,0,J,11,161883
442,28K3T,717,hight,0,T,10,167226
...,...,...,...,...,...,...,...
322,AAA8A,887,four,5,A,14,5978574
83,AAAA5,114,four,5,A,14,5978661
381,AAAA7,303,four,5,A,14,5978663
149,AAAAJ,690,four,5,J,11,5978667


In [7]:
sum([(ix+1) *v for ix, v in enumerate(sorted_df['bid'])])

250957639

## Part 2

In [8]:
card = {v: k for k, v in enumerate(list("_J23456789TQKA"))}
del card["_"]
card

{'J': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'T': 10,
 'Q': 11,
 'K': 12,
 'A': 13}

In [9]:
def update_joker(hand: str):
  if 'J' not in hand: 
    return hand
  c = pd.DataFrame(Counter(list(hand)).items()).sort_values(1)
  f_c = c[c[0] != "J"]
  max_card = f_c.max()[0]

  if pd.isna(max_card):
    return "AAAAA"
  return hand.replace('J', max_card)



In [10]:
df['j_hand']= df['hand'].apply(update_joker)
df

,hand,bid,type,type_val,high,high_val,score,j_hand
0,398KA,456,hight,0,K,13,235742,398KA
1,2J299,282,pair_2,2,J,11,2176793,29299
2,8939K,547,pair_1,1,K,13,1562077,8939K
3,9TAA9,79,pair_2,2,T,10,2634601,9TAA9
4,47TJ4,431,pair_1,1,T,10,1293556,47TT4
...,...,...,...,...,...,...,...,...
995,TJKQK,260,pair_1,1,T,10,1703949,TTKQK
996,44945,374,three,3,9,9,3280901,44945
997,K2AT3,313,hight,0,T,10,863907,K2AT3
998,33363,311,four,5,6,6,5209763,33363


In [11]:
df['j_type'] = pd.DataFrame(df.apply(lambda r: get_hand_type(r["j_hand"]), axis=1))
df['j_type_val'] = df.apply(lambda r: types.get(r['j_type']), axis=1)
df['j_high'] = pd.DataFrame(df.apply(lambda r: get_high_card(r["j_hand"]), axis=1))
df['j_high_val'] = df.apply(lambda r: card.get(r['j_high']), axis=1)
df['j_score'] = df.apply(lambda r: get_card_score(r['hand'], r['j_type']), axis=1)

df

,hand,bid,type,type_val,high,high_val,score,j_hand,j_type,j_type_val,j_high,j_high_val,j_score
0,398KA,456,hight,0,K,13,235742,398KA,hight,0,K,12,235725
1,2J299,282,pair_2,2,J,11,2176793,29299,full,4,9,9,4135833
2,8939K,547,pair_1,1,K,13,1562077,8939K,pair_1,1,K,12,1562076
3,9TAA9,79,pair_2,2,T,10,2634601,9TAA9,pair_2,2,T,10,2634329
4,47TJ4,431,pair_1,1,T,10,1293556,47TT4,pair_2,2,T,10,2293396
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,TJKQK,260,pair_1,1,T,10,1703949,TTKQK,pair_2,2,T,10,2662716
996,44945,374,three,3,9,9,3280901,44945,three,3,9,9,3280901
997,K2AT3,313,hight,0,T,10,863907,K2AT3,hight,0,T,10,798115
998,33363,311,four,5,6,6,5209763,33363,four,5,6,6,5209763


In [12]:
sorted_df = df.sort_values('j_score')
sorted_df

,hand,bid,type,type_val,high,high_val,score,j_hand,j_type,j_type_val,j_high,j_high_val,j_score
156,265K8,511,hight,0,K,13,157144,265K8,hight,0,K,12,157128
672,26957,462,hight,0,9,9,158039,26957,hight,0,9,9,158039
442,28K3T,717,hight,0,T,10,167226,28K3T,hight,0,T,10,166970
391,293TA,789,hight,0,T,10,168878,293TA,hight,0,T,10,168877
454,2946T,338,hight,0,T,10,169066,2946T,hight,0,T,10,169066
...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,TTTJT,315,four,5,T,10,5699066,TTTTT,five,6,T,10,6698906
5,KJKKK,262,four,5,K,13,5900573,KKKKK,five,6,K,12,6793804
145,KKKJJ,971,full,4,K,13,4908731,KKKKK,five,6,K,12,6838673
754,AJAAJ,235,full,4,J,11,4966379,AAAAA,five,6,A,13,6859601


In [13]:
sum([(ix+1) *v for ix, v in enumerate(sorted_df['bid'])])

251114695